In [0]:
from pyspark.sql.functions import col, current_timestamp, coalesce, lit

# Carga de tablas
df_compras = spark.read.table("workspace.silver.compras")
dim_prov = spark.read.table("workspace.gold.dim_proveedores")
dim_prod = spark.read.table("workspace.gold.dim_productos")
dim_tiempo = spark.read.table("workspace.gold.dim_tiempo")

# 1. Definimos la clave por defecto
DEFAULT_KEY = -1 

# 2. Cruces y selección (Aplicando la lógica de "Miembro Desconocido")
fact_compras = df_compras.join(dim_prod, df_compras.codigo_producto == dim_prod.codigo_producto_bk, "left") \
    .join(dim_prov, df_compras.ruc == dim_prov.ruc_bk, "left") \
    .join(dim_tiempo, df_compras.fecha == dim_tiempo.fecha_bk, "left") \
    .select(
        # Limpieza de SKs
        coalesce(col("tiempo_key"), lit(DEFAULT_KEY)).alias("tiempo_key"),
        coalesce(col("producto_key"), lit(DEFAULT_KEY)).alias("producto_key"),
        coalesce(col("proveedor_key"), lit(DEFAULT_KEY)).alias("proveedor_key"),
        
        # Atributos de la transacción
        col("numero_compra"),
        col("cantidad"),
        col("costo_unitario"),
        col("precio_unitario"),
        col("subtotal"),
        col("iva"),
        col("total"),
        
        # Auditoría
        current_timestamp().alias("fecha_carga")
    )

# 3. Escritura en la Capa Gold
fact_compras.write.mode("overwrite").saveAsTable("workspace.gold.fact_compras")

print("✅ fact_compras creada con manejo de nulos.")